# Peer-graded Assignment: Analyzing Historical Stock/Revenue Data and Building a Dashboard

I am doing the project in my own Jupyter Notebook. So kindly be generous with your scoring. 

In [1]:
# Importing the libraries

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Creating the make_graph function. It's a copy-pase from the lab.

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# 1. Tesla

## Stock data

In [20]:
# Extracting the TESLA Information from the yf library.

tesla = yf.Ticker('TSLA')

# Getting the stock history data and assigning into a new dataframe called tesla_data
tesla_data = tesla.history(period='max')

# Reseting the index of our dataframe.

tesla_data.reset_index(inplace=True)

# Printing the first 5 rows of the dataframe.

tesla_data.head().round(2)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.27,1.67,1.17,1.59,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.72,2.03,1.55,1.59,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.67,1.73,1.35,1.46,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.53,1.54,1.25,1.28,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.33,1.33,1.06,1.07,103003500,0.0,0.0


## Web Scraping

In [17]:
# Assiging the URL to a variable called url.
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'

# Assigning the response data to new variable called html_data
html_data = requests.get(url).text

beautiful_soup = BeautifulSoup(html_data, "html.parser")

# Extracting the table data using pandas read_html
tables = pd.read_html(html_data)
test_table = tables[0]

# Creating a new df with new columns:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

# Assigning the values to the right columns
tesla_revenue['Date'] = pd.to_datetime(test_table['Tesla Annual Revenue (Millions of US $)'])
tesla_revenue['Revenue'] = test_table['Tesla Annual Revenue (Millions of US $).1']

# Changing the datetypes of the Date column
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',|\$',"")

# Droping the missing values and empty rows
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

## Showing Dashboard

In [16]:
# Making graph of Tesla.
make_graph(tesla_data, tesla_revenue, 'Tesla')

# 2. GameStop

## Stock data

In [19]:
# Extracting GME's info
gme = yf.Ticker('GME')

# Finding GME's stock history

gme_data = gme.history(period='max')

# Resetting the index of the dataframe.

gme_data.reset_index(inplace=True)

# printing the first 5 rows.

gme_data.head().round(2)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.62,1.69,1.60,1.69,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.71,1.72,1.67,1.68,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.68,1.69,1.66,1.67,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.67,1.67,1.58,1.61,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.62,1.66,1.60,1.66,6892800,0.0,0.0


## Web Scraping

In [22]:
# Assigning the url to a new variable
gme_html = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'

# Assigning the retrieved data to a variable.
import requests
html_data = requests.get(gme_html).text

# Scraping 
from bs4 import BeautifulSoup
beautiful_soup = BeautifulSoup(html_data, 'html5lib')

# Extracting the table data using pandas read_html
import pandas as pd
tables = pd.read_html(html_data)

# Assuming the table we want is the first one, we can access it using index 0
test_tab = tables[0]

# Creating a new df with new columns:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

# Assigning the values to the right columns
gme_revenue['Date'] = pd.to_datetime(test_tab['GameStop Annual Revenue (Millions of US $)'])
gme_revenue['Revenue'] = test_tab['GameStop Annual Revenue (Millions of US $).1']

# Removing the comma and dollar sign.
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',|\$',"")

# Dropping the missing values and empty rows
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

## Showing Dashboar

In [23]:
# Making graph for Gamestop.
make_graph(gme_data, gme_revenue, 'GameStop')